In [2]:
import os
import numpy as np
import pandas as pd
import sys
import tqdm

sys.path.append('..')
from anomalies.increasing import *
from anomalies.invariant import *
from anomalies.decreasing import *

from detectors.feature_based import *
from detectors.pca_based import *
from detectors.gaussian_distribution_based import *
from detectors.ml_based import *
from datasets.data_utils import load_data_training, load_data_test

import matplotlib.pyplot as plt
%matplotlib widget
import cmasher as cmr
cmap = cmr.get_sub_cmap('brg', 0.5, 1)
import dataframe_image as dfi

ModuleNotFoundError: No module named 'wombats'

## Setup

In [4]:
data_type = 'bridge'
delta = 0.02
metric = 'P_D'
  
N_train = None
gamma = 'scale'

if data_type == 'ecg':
    channels = ['z']
    n = len(channels)*256
    SNRdB = 40
    
    k1 = 52
    k2 = 59
    
    p1 = 4
    p2 = 16
    
    nu = 0.01
    kernel = 'poly'
    degree = 3
    k1 = 52
    
    h = 5
    l = 250
    
elif data_type == 'bridge':
    channels = ['x']
    n = len(channels)*100
    SNRdB = None

    k1 = 16
    k2 = 85

    p1 = 4
    p2 = 16

    nu = 0.01
    kernel = 'rbf'

    h = 5
    l = 500
    



## Load train data

In [5]:
Xok_train, Xok_val = load_data_training(channels, n, data_type)
Xok_train = Xok_train[..., -1]
Xok_val = Xok_val[..., -1]

## Train detectors

In [5]:
detectors_labels = [f'SPE_{k1}', f'SPE_{k2}', f'T2_{k1}', f'T2_{k2}', f'AR_{p1}', f'AR_{p2}', 'MD', f'OCSVM_{kernel}_{nu}', f'LOF_{h}',f'IF_{l}', 'energy', 'TV', 'ZC', 'pk-pk']
detectors = [SPE(k1), SPE(k2), T2(k1), T2(k2), AR(p1), AR(p2), MD(), OCSVM(kernel, nu), LOF(h), IF(l), energy(), TV(), ZC(), pk_pk()]
detectors_dict = dict(zip(detectors_labels, detectors))

for detector_label, detector in tqdm.tqdm(detectors_dict.items()):
    detectors_dict[detector_label] = detector.fit(Xok_train[:N_train])

100%|██████████| 14/14 [00:28<00:00,  2.03s/it]


## Load test data

In [6]:
Xok_test = load_data_test(channels, n, data_type)[..., -1]

## Generate Anomalies

In [7]:
anomalies_labels = [
    'GWN', 'Impulse', 'Step', 'Constant',  
    'GNN',
    'MixingGWN', 'MixingConstant',
    'SpectralAlteration', 'PrincipalSubspaceAlteration',
    'TimeWarping',
    'Clipping', 'Dead-Zone'
]

# # create anomalies class instance
anomalies = [
    GWN(delta), Impulse(delta), Step(delta), Constant(delta),  
    GNN(delta),
    MixingGWN(delta), MixingConstant(delta),
    SpectralAlteration(delta), PrincipalSubspaceAlteration(delta),
    TimeWarping(delta), 
    Clipping(delta), DeadZone(delta)
]

anomalies_dict = dict(zip(anomalies_labels, anomalies))

# initialize the dataframe for anomalous data
Xko_df = pd.DataFrame(
    index=np.arange(Xok_test.shape[0]),
    columns=pd.MultiIndex.from_product([anomalies_labels, np.arange(Xok_test.shape[1])])
)

# generate anomalous data for each anomaly
for anomaly_label, anomaly in tqdm.tqdm(anomalies_dict.items()):
    if anomaly_label in ['SpectralAlteration']:
        anomaly.fit(Xok_test, SNRdB)
    else:
        anomaly.fit(Xok_test)
    # Xko_df.loc[0, name] = anomaly.distort(ECG_test.values[0])
    Xko_df[anomaly_label] = anomaly.distort(Xok_test)

100%|██████████| 12/12 [03:10<00:00, 15.88s/it]


## Evaluate detectors

In [8]:
result = pd.DataFrame(index=anomalies_labels, columns=detectors_labels, dtype=np.float64)

for detector_label, detector in tqdm.tqdm(detectors_dict.items()):
    for anomaly_label in anomalies_labels:
        Xko_test = Xko_df[anomaly_label].values
        X_anom = np.concatenate([Xok_test, Xko_test])
        metric_value = detector.test(X_anom, metric)
        result.loc[anomaly_label, detector_label] = np.round(metric_value, 2)

result.style.background_gradient(cmap=cmap, vmin=0.5, vmax=1)

100%|██████████| 14/14 [02:04<00:00,  8.92s/it]


,SPE_52,SPE_59,T2_52,T2_59,AR_4,AR_16,MD,OCSVM_poly_0.01,LOF_5,IF_250,energy,TV,ZC,pk-pk
GWN,1.000000,1.000000,0.520000,0.550000,1.000000,1.000000,1.000000,0.500000,0.850000,0.540000,0.520000,1.000000,0.990000,0.610000
Impulse,1.000000,1.000000,0.520000,0.550000,1.000000,1.000000,1.000000,0.500000,0.860000,0.520000,0.520000,0.690000,0.650000,0.670000
Step,0.520000,0.540000,0.570000,0.550000,0.790000,0.910000,0.910000,0.500000,0.710000,0.530000,0.520000,0.510000,0.510000,0.550000
Constant,0.500000,0.500000,0.510000,0.510000,0.500000,0.500000,0.500000,0.500000,0.560000,0.530000,0.520000,0.500000,0.520000,0.500000
GNN,0.920000,0.920000,0.520000,0.550000,0.920000,0.920000,0.920000,0.500000,0.840000,0.540000,0.520000,0.920000,0.900000,0.600000
MixingGWN,1.000000,1.000000,0.500000,0.540000,1.000000,1.000000,1.000000,0.540000,0.840000,0.520000,0.500000,1.000000,0.990000,0.570000
MixingConstant,0.510000,0.510000,0.510000,0.510000,0.520000,0.520000,0.530000,0.540000,0.550000,0.510000,0.500000,0.510000,0.510000,0.540000
SpectralAlteration,0.500000,0.500000,0.520000,0.520000,0.510000,0.510000,0.510000,0.540000,0.830000,0.510000,0.500000,0.620000,0.630000,0.510000
PrincipalSubspaceAlteration,1.000000,1.000000,0.500000,0.540000,1.000000,1.000000,1.000000,0.540000,0.850000,0.530000,0.500000,1.000000,0.980000,0.570000
TimeWarping,0.510000,0.520000,0.510000,0.510000,0.680000,0.710000,0.700000,0.500000,0.510000,0.500000,0.500000,0.510000,0.510000,0.500000


## Save

In [9]:
path = os.path.join('results', f'results-data={data_type}-n={n}-delta={delta}.pkl')
result.to_pickle(path)